In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
pass_big = pd.read_csv('/kaggle/input/passing-nba/nba passing bigs - BasketballReference Stats.csv')
pass_adv = pd.read_csv('/kaggle/input/passing-nba/nba passing bigs - NBA Stats.csv')

print(pass_big.head())
print(pass_adv.head())

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/passing-nba/nba passing bigs - NBA Stats.csv
/kaggle/input/passing-nba/nba passing bigs - BasketballReference Stats.csv
   Rk             Player Pos  Age   Tm   G    MP  PG%  SG%  SF%  ... On-Off  \
0   1  Tyrese Haliburton  PG   23  IND  69  2224  90%  10%  NaN  ...    5.7   
1   2   Domantas Sabonis   C   27  SAC  82  2928  NaN  NaN  NaN  ...   -2.1   
2   3       Nikola Jokic   C   28  DEN  79  2737  NaN  NaN  NaN  ...   20.0   
3   4        Luka Dončić  PG   24  DAL  70  2624  74%  26%  NaN  ...    9.4   
4   5       James Harden  PG   34  LAC  72  2470  72%  28%  NaN  ...    6.0   

  BadPass  LostBall  Shoot  Off.  Shoot.1  Off..1  PGA\n▼  And1  Blkd  
0     121        19     49     5       97       7    1782    42    57  
1      90       100    123    48      210       7    1679    56    86  
2     143        66     97    21      181       5    1662    59    68  
3     200        61     77     7      283      12    1636    78    50  
4     120        42     62    1

In [2]:
new_col = pass_big[['Player', 'LostBall', 'BadPass']]

passing_big = pd.merge(pass_adv, new_col, left_on='PLAYER', right_on='Player', how='left')

passing_big = passing_big.drop(columns=['Player'])

print(passing_big)
print(passing_big.columns)

                PLAYER TEAM  GP   W   L   MIN  Passes\nMade  Passes\nReceived  \
0         Nikola Jokic  DEN  77  53  24  2670          5768              5225   
1     Domantas Sabonis  SAC  82  46  36  2928          5843              4915   
2         Kelly Olynyk  TOR  77  31  46  1726          2851              2281   
3       Alperen Sengun  HOU  63  29  34  2046          2490              2824   
4         Jusuf Nurkic  PHX  75  48  27  2050          3188              2436   
5          Bam Adebayo  MIA  71  40  31  2416          3026              2973   
6    Victor Wembanyama  SAS  71  19  52  2106          2462              2812   
7        Anthony Davis  LAL  75  44  31  2669          3771              3274   
8       Nikola Vucevic  CHI  75  36  39  2574          3382              3402   
9        Julius Randle  NYK  46  29  17  1630          2116              2446   
10         Joel Embiid  PHI  39  31   8  1309          1658              2194   
11       Jarrett Allen  CLE 

In [3]:
# Cumulative score formula (simplified)

passing_big['tot_score'] = (0.40 * passing_big['AST'] + 
                          0.10 * passing_big['Secondary\nAST'] + 
                          0.05 * passing_big['Potential\nAST'] + 
                          0.15 * passing_big['Passes\nMade'] + 
                          0.15 * passing_big['AST PTS\nCreated'] - 
                          0.15 * (passing_big['LostBall'] + passing_big['BadPass']))

top_pass_big = passing_big.nlargest(10, 'tot_score')

# Display the result
print(top_pass_big)

                PLAYER TEAM  GP   W   L   MIN  Passes\nMade  Passes\nReceived  \
0         Nikola Jokic  DEN  77  53  24  2670          5768              5225   
1     Domantas Sabonis  SAC  82  46  36  2928          5843              4915   
7        Anthony Davis  LAL  75  44  31  2669          3771              3274   
2         Kelly Olynyk  TOR  77  31  46  1726          2851              2281   
5          Bam Adebayo  MIA  71  40  31  2416          3026              2973   
12       Chet Holmgren  OKC  82  57  25  2413          3218              2464   
3       Alperen Sengun  HOU  63  29  34  2046          2490              2824   
6    Victor Wembanyama  SAS  71  19  52  2106          2462              2812   
11       Jarrett Allen  CLE  77  46  31  2442          2679              2426   
15  Isaiah Hartenstein  NYK  75  47  28  1896          2525              1582   

    AST  Secondary\nAST  Potential\nAST  AST PTS\nCreated  AST\nAdj  \
0   693              79            11